<a href="https://colab.research.google.com/github/Abanoub1111/GBG_TASKS/blob/main/GBG_s1_(Text_processing_%26_Sentiment_analysis_using_RNN%2C_LSTM_and_GRU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
saurabhshahane_arabic_classification_path = kagglehub.dataset_download('saurabhshahane/arabic-classification')

print('Data source import complete.')

100%|██████████| 83.4M/83.4M [00:00<00:00, 110MB/s]

Extracting files...


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(saurabhshahane_arabic_classification_path)

/root/.cache/kagglehub/datasets/saurabhshahane/arabic-classification/versions/1


Here are the files inside the data directory:

In [ ]:
import os
for dirname, _, filenames in os.walk(saurabhshahane_arabic_classification_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/root/.cache/kagglehub/datasets/saurabhshahane/arabic-classification/versions/1/arabic_dataset_classifiction.csv/arabic_dataset_classifiction.csv


,text,targe
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0


In [ ]:
print(df.loc[1, 'text'])

قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر عملها على الفن بل عملت مع أحد المتخصصين لإطلاق نوع جديد من الشاي سيصبح متوفرا ابتداء من الشهر المقبل في سلسلة مقاهي ستاربكس ونقلت وسائل إعلام أمريكية عن رئيس مجلس إدارة ستاربكس هاورد شولتز ووينفري إعلانهما عن ابتكار نوع جديد من الشاي يحمل اسم الذي سيباع ابتداء من أبريل نيسان المقبل في مقاهي ستاربكس وتيفانا بأمريكا وكندا وتعتزم ستاربكس التبرع بعائدات بيع هذا الشاي لأكاديمية أسستها وينفري وتعنى بتوفير فرص تعليم للشبان


In [ ]:
print("First 5 rows of 'text' column:")
display(df['text'].head())

print("\nMissing values in 'text' column:")
print(df['text'].isnull().sum())

print("\nLength of example texts (index 0, 1, 2):")
print(f"Index 0 length: {len(df.loc[0, 'text'])}")
print(f"Index 1 length: {len(df.loc[1, 'text'])}")
print(f"Index 2 length: {len(df.loc[2, 'text'])}")

print("\nRandomly sampled 5 entries from 'text' column for manual inspection:")
display(df['text'].sample(5, random_state=42))

First 5 rows of 'text' column:


,text
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...



Missing values in 'text' column:
2939

Length of example texts (index 0, 1, 2):
Index 0 length: 1890
Index 1 length: 464
Index 2 length: 939

Randomly sampled 5 entries from 'text' column for manual inspection:


,text
15978,تمكنت مصالح الدرك بمركز السهول ضواحي سلا من تف...
50440,عبر نبيل بن عبد الله أمين عام حزب التقدم والاش...
9663,يقوم وفد من اتحاد الكتاب الصينيين مكون من ستة ...
86848,أفاد مصدر مسؤول أن أولمبيك آسفي لكرة القدم قرر...
53415,عبدالاله بوسحابة اخبارنا المغربية ذكرت وكالة أ...


In [ ]:
print(f"Original number of rows: {len(df)}")
df.dropna(subset=['text'], inplace=True)
print(f"Number of rows after dropping missing values: {len(df)}")
print(f"Missing values in 'text' column after dropping: {df['text'].isnull().sum()}")

Original number of rows: 111728
Number of rows after dropping missing values: 108789
Missing values in 'text' column after dropping: 0


In [ ]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove non-Arabic characters and non-spaces
    text = re.sub(r'[^\u0621-\u064A\s]+', '', text)
    # Remove digits (if any remain after previous step, or if previous step was adjusted)
    text = re.sub(r'\d+', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove leading and trailing spaces
    text = text.strip()
    return text

# Apply the clean_text function to the 'text' column
df['text'] = df['text'].apply(clean_text)

print("First 5 rows of 'text' column after cleaning:")
display(df['text'].head())

First 5 rows of 'text' column after cleaning:


,text
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...


In [ ]:
import nltk
# No need for nltk.download('punkt') here, as the previous cell already confirmed it's up-to-date,
# and the error is specifically about 'punkt_tab/arabic/', which isn't directly resolved by downloading 'punkt' again.

# Apply simple whitespace tokenization to the 'text' column to avoid the LookupError
df['tokenized_text'] = df['text'].apply(lambda x: x.split())

print("First 5 rows of 'tokenized_text' column after tokenization (using whitespace split):")
for i in range(5):
    print(f"Row {i}: {df['tokenized_text'].iloc[i]}")

First 5 rows of 'tokenized_text' column after tokenization (using whitespace split):
Row 0: ['بين', 'أستوديوهات', 'ورزازات', 'وصحراء', 'مرزوكة', 'وآثار', 'وليلي', 'ثم', 'الرباط', 'والبيضاء', 'انتهى', 'المخرج', 'المغربي', 'سهيل', 'بن', 'بركة', 'من', 'تصوير', 'مشاهد', 'عمله', 'السينمائي', 'الجديد', 'الذي', 'خصصه', 'لتسليط', 'الضوء', 'عن', 'حياة', 'الجاسوس', 'الإسباني', 'دومينغو', 'باديا', 'الذي', 'عاش', 'فترة', 'من', 'القرن', 'التاسع', 'عشر', 'بالمغرب', 'باسم', 'علي', 'باي', 'هذا', 'الفيلم', 'الذي', 'اختار', 'له', 'مخرجه', 'عنوان', 'حلم', 'خليفة', 'يصور', 'حياة', 'علي', 'باي', 'العباسي', 'الذي', 'ما', 'زال', 'أحد', 'أحياء', 'طنجة', 'يحمل', 'اسمه', 'عاش', 'حياة', 'فريدة', 'متنكرا', 'بشخصية', 'تاجر', 'عربي', 'من', 'سلالة', 'الرسول', 'صلى', 'الله', 'عليه', 'وسلم', 'فيما', 'كان', 'يعمل', 'جاسوسا', 'لحساب', 'إسبانيا', 'وكشف', 'مخرج', 'الفيلم', 'سهيل', 'بن', 'بركة', 'في', 'تصريح', 'لهسبريس', 'أن', 'الفيلم', 'السينمائي', 'دخل', 'مرحلة', 'التوضيب', 'التي', 'تتم', 'خارج', 'المغرب', 'مبرزا', 'أن',

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

print(arabic_stopwords)

{'مارس', 'سادس', 'ل', 'لئن', 'هَجْ', 'بؤسا', 'ذات', 'جيم', 'ت', 'ذ', 'ذلكما', 'اللذان', 'فيما', 'إياه', 'له', 'لهما', 'الألى', 'تلقاء', 'هكذا', 'بسّ', 'الآن', 'لعلَّ', 'ثلاثين', 'أنًّ', 'حيَّ', 'ذاك', 'دال', 'تسع', 'حاشا', 'تاسع', 'منه', 'ولو', 'آمينَ', 'إياي', 'آهٍ', 'حادي', 'نيسان', 'إذما', 'لنا', 'أخٌ', 'سبعين', 'غير', 'كى', 'بما', 'ديسمبر', 'بكن', 'اثني', 'حيثما', 'هما', 'بك', 'آهاً', 'ءَ', 'تفعلين', 'آب', 'بهما', 'إليكنّ', 'درهم', 'أول', 'خمسة', 'قطّ', 'ث', 'فلس', 'هناك', 'اتخذ', 'لمّا', 'أمسى', 'م', 'لعمر', 'بكم', 'ع', 'تارة', 'خمسمائة', 'غالبا', 'ذين', 'عوض', 'ب', 'آ', 'أنشأ', 'أى', 'إياك', 'رجع', 'ظ', 'شتان', 'مذ', 'شين', 'به', 'فضلا', 'تلك', 'ثمانية', 'أحد', 'ستون', 'ح', 'هنا', 'أربعاء', 'ء', 'حين', 'و', 'ظاء', 'تِي', 'ها', 'إليكَ', 'أكتوبر', 'حسب', 'ضاد', 'صبرا', 'قرش', 'أينما', 'أل', 'أعطى', 'بمن', 'وَيْ', 'أين', 'هاكَ', 'عجبا', 'كأن', 'إنه', 'بماذا', 'أنتِ', 'لا', 'آي', 'الذي', 'ثم', 'ليستا', 'كاف', 'تموز', 'خمسمئة', 'ذي', 'هاك', 'كيف', 'منها', 'هاتان', 'حاي', 'مكانَك', 'وا

In [ ]:
# Load Arabic stop words
arabic_stopwords = set(stopwords.words('arabic'))

# Function to remove stop words
def remove_stopwords(tokens):
    return [word for word in tokens if word not in arabic_stopwords]

# Apply stop word removal to the tokenized_text column
df['filtered_text'] = df['tokenized_text'].apply(remove_stopwords)

print("First 5 rows of 'filtered_text' column after stop word removal:")
for i in range(5):
    print(f"Row {i}: {df['filtered_text'].iloc[i]}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


First 5 rows of 'filtered_text' column after stop word removal:
Row 0: ['أستوديوهات', 'ورزازات', 'وصحراء', 'مرزوكة', 'وآثار', 'وليلي', 'الرباط', 'والبيضاء', 'انتهى', 'المخرج', 'المغربي', 'سهيل', 'بن', 'بركة', 'تصوير', 'مشاهد', 'عمله', 'السينمائي', 'الجديد', 'خصصه', 'لتسليط', 'الضوء', 'حياة', 'الجاسوس', 'الإسباني', 'دومينغو', 'باديا', 'عاش', 'فترة', 'القرن', 'التاسع', 'بالمغرب', 'باسم', 'علي', 'باي', 'الفيلم', 'اختار', 'مخرجه', 'عنوان', 'حلم', 'خليفة', 'يصور', 'حياة', 'علي', 'باي', 'العباسي', 'زال', 'أحياء', 'طنجة', 'يحمل', 'اسمه', 'عاش', 'حياة', 'فريدة', 'متنكرا', 'بشخصية', 'تاجر', 'عربي', 'سلالة', 'الرسول', 'صلى', 'الله', 'وسلم', 'يعمل', 'جاسوسا', 'لحساب', 'إسبانيا', 'وكشف', 'مخرج', 'الفيلم', 'سهيل', 'بن', 'بركة', 'تصريح', 'لهسبريس', 'الفيلم', 'السينمائي', 'دخل', 'مرحلة', 'التوضيب', 'تتم', 'خارج', 'المغرب', 'مبرزا', 'الفيلم', 'يروي', 'حياة', 'الجاسوس', 'الإسباني', 'دومينغو', 'باديا', 'قرر', 'طنجة', 'بدء', 'رحلاته', 'عدد', 'المناطق', 'العالم', 'الإسلامي', 'بداية', 'القرن', 'العشرين', '

In [ ]:
import nltk
from nltk.stem.isri import ISRIStemmer

# Initialize the ISRI Stemmer
stemmer = ISRIStemmer()

# Define a function to apply stemming to a list of tokens
def arabic_stemmer(tokens):
    return [stemmer.stem(word) for word in tokens]

# Apply the stemming function to the 'filtered_text' column
df['stemmed_text'] = df['filtered_text'].apply(arabic_stemmer)

print("First 5 rows of 'stemmed_text' column after stemming:")
for i in range(5):
    print(f"Row {i}: {df['stemmed_text'].iloc[i]}")

First 5 rows of 'stemmed_text' column after stemming:
Row 0: ['استوديوه', 'رزز', 'صحراء', 'رزك', 'آثر', 'ولل', 'ربط', 'يضء', 'نهى', 'خرج', 'غرب', 'سهل', 'بن', 'برك', 'صور', 'شهد', 'عمل', 'نمئ', 'جدد', 'خصص', 'سلط', 'ضوء', 'حية', 'جسس', 'اسب', 'دومينغو', 'باد', 'عاش', 'فتر', 'قرن', 'تسع', 'غرب', 'بسم', 'علي', 'باي', 'يلم', 'خار', 'خرج', 'عنو', 'حلم', 'خلف', 'يصر', 'حية', 'علي', 'باي', 'عبس', 'زال', 'حيء', 'طنج', 'حمل', 'اسم', 'عاش', 'حية', 'فرد', 'تنكر', 'شخص', 'تجر', 'عرب', 'سلل', 'رسل', 'صلى', 'الل', 'سلم', 'عمل', 'جسس', 'حسب', 'سبن', 'كشف', 'خرج', 'يلم', 'سهل', 'بن', 'برك', 'صرح', 'هسبريس', 'يلم', 'نمئ', 'دخل', 'رحل', 'وضب', 'تتم', 'خرج', 'غرب', 'برز', 'يلم', 'يري', 'حية', 'جسس', 'اسب', 'دومينغو', 'باد', 'قرر', 'طنج', 'بدء', 'رحل', 'عدد', 'نطق', 'علم', 'سلم', 'بدي', 'قرن', 'عشر', 'سيك', 'جهز', 'شهر', 'جمع', 'يلم', 'نمئ', 'عدد', 'مثل', 'خلف', 'جنس', 'خار', 'لدر', 'بطل', 'مثل', 'سينمائية', 'يطل', 'كرل', 'كرش', 'قيم', 'بدر', 'انجليزية', 'ليد', 'هستر', 'هوب', 'شهر', 'كتب', 'غرب', 'بـ', '

## Prepare Data for Embedding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Join tokens back into strings for the tokenizer
df['stemmed_text_joined'] = df['stemmed_text'].apply(lambda x: ' '.join(x))

# Initialize Tokenizer
# Limiting vocabulary to top 10000 words
MAX_NUM_WORDS = 10000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token='<unk>')

# Fit tokenizer on the joined stemmed text
tokenizer.fit_on_texts(df['stemmed_text_joined'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['stemmed_text_joined'])

print("Tokenizer fitted and text converted to sequences. First 5 sequences:")
for i in range(5):
    print(f"Row {i}: {sequences[i]}")
print(f"Total unique tokens found by tokenizer: {len(tokenizer.word_index)}")

Tokenizer fitted and text converted to sequences. First 5 sequences:
Row 0: [5176, 1746, 547, 8858, 4383, 1926, 76, 190, 1090, 66, 2, 525, 568, 462, 236, 119, 6, 1013, 22, 78, 212, 969, 567, 4996, 444, 1, 7229, 1065, 186, 470, 1223, 2, 705, 223, 4554, 403, 687, 66, 742, 807, 65, 2311, 567, 223, 4554, 1776, 1181, 1091, 288, 98, 479, 1065, 567, 344, 1, 100, 574, 111, 2033, 294, 5081, 272, 82, 6, 4996, 96, 457, 139, 66, 403, 525, 568, 462, 145, 484, 403, 1013, 21, 123, 6871, 1002, 66, 2, 255, 403, 3520, 567, 4996, 444, 1, 7229, 28, 288, 844, 123, 19, 105, 10, 82, 237, 470, 450, 618, 313, 99, 3, 403, 1013, 19, 103, 65, 570, 687, 1549, 51, 103, 1336, 529, 2412, 2108, 165, 242, 2029, 3961, 1, 4782, 99, 56, 2, 346, 1, 260, 2388, 242, 24, 111, 76, 4673, 4554, 72, 1397, 205, 89, 1336, 60, 800, 94, 66, 2, 2768, 223, 4554, 1776, 228, 145, 170, 1353, 125, 100, 2388, 46, 2465, 224, 798, 212, 232, 969, 281, 403, 98, 224, 2827, 657, 51, 527, 1405, 1337, 1, 4626, 13, 123, 1951, 164, 100, 808, 468, 274

In [ ]:
import numpy as np

# Calculate sequence lengths
sequence_lengths = [len(seq) for seq in sequences]

# Determine maxlen using a percentile (e.g., 90th percentile)
maxlen = int(np.percentile(sequence_lengths, 90))

print(f"Calculated 90th percentile sequence length (maxlen): {maxlen}")

# Pad sequences to the determined maxlen
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

print(f"Shape of padded sequences: {padded_sequences.shape}")
print("First 5 padded sequences (truncated for brevity):")
for i in range(5):
    print(f"Row {i}: {padded_sequences[i][:10]}...") # Print first 10 elements for brevity

Calculated 90th percentile sequence length (maxlen): 369
Shape of padded sequences: (108789, 369)
First 5 padded sequences (truncated for brevity):
Row 0: [5176 1746  547 8858 4383 1926   76  190 1090   66]...
Row 1: [  28  314   37 1358 1761  347    6  372    6  990]...
Row 2: [  85    2 1653  236 6628 3768  314    2 3696 1802]...
Row 3: [  85    2  118 1388  267  755   68  406  598  170]...
Row 4: [1964  247 2336    2   95  109  711 1619  227  645]...


## Split Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and labels (y)
X = padded_sequences
y = df['targe']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (87031, 369)
Shape of X_test: (21758, 369)
Shape of y_train: (87031,)
Shape of y_test: (21758,)


## Build and Train RNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# 2. Determine vocabulary size
vocabulary_size = len(tokenizer.word_index) + 1

# 3. Determine number of classes
num_classes = y_train.nunique()

# 4. Define embedding dimension and RNN units
embedding_dim = 100
rnn_units = 64

# 5. Create a Sequential model
model = Sequential()
# a. Add an Embedding layer
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim))
# b. Add a SimpleRNN layer
model.add(SimpleRNN(units=rnn_units))
# c. Add a Dense output layer
model.add(Dense(units=num_classes, activation='softmax'))

# 6. Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 7. Print the model summary
model.summary()

print(f"Vocabulary Size: {vocabulary_size}")
print(f"Number of Classes: {num_classes}")
print(f"Embedding Dimension: {embedding_dim}")
print(f"RNN Units: {rnn_units}")

# 8. Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
print("Model training complete.")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Vocabulary Size: 81848
Number of Classes: 5
Embedding Dimension: 100
RNN Units: 64
Epoch 1/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 86s 30ms/step - accuracy: 0.3935 - loss: 1.5001 - val_accuracy: 0.4147 - val_loss: 1.4727
Epoch 2/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 82s 30ms/step - accuracy: 0.4539 - loss: 1.3958 - val_accuracy: 0.4257 - val_loss: 1.4574
Epoch 3/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 81s 30ms/step - accuracy: 0.4409 - loss: 1.4225 - val_accuracy: 0.4389 - val_loss: 1.4182
Epoch 4/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 81s 30ms/step - accuracy: 0.4501 - loss: 1.3970 - val_accuracy: 0.4408 - val_loss: 1.4240
Epoch 5/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 81s 30ms/step - accuracy: 0.4593 - loss: 1.3803 - val_accuracy: 0.4451 - val_loss: 1.4211
Model training complete.


## Build and Train LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Vocabulary & classes
vocabulary_size = len(tokenizer.word_index) + 1
num_classes = y_train.nunique()

embedding_dim = 100
rnn_units = 64

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim))
model.add(LSTM(units=rnn_units))
model.add(Dense(units=num_classes, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test)
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 61s 20ms/step - accuracy: 0.4809 - loss: 1.2997 - val_accuracy: 0.7091 - val_loss: 0.6980
Epoch 2/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 54s 20ms/step - accuracy: 0.8557 - loss: 0.4398 - val_accuracy: 0.9439 - val_loss: 0.1887
Epoch 3/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 54s 20ms/step - accuracy: 0.9528 - loss: 0.1623 - val_accuracy: 0.9518 - val_loss: 0.1573
Epoch 4/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 54s 20ms/step - accuracy: 0.9658 - loss: 0.1161 - val_accuracy: 0.9535 - val_loss: 0.1552
Epoch 5/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 54s 20ms/step - accuracy: 0.9738 - loss: 0.0899 - val_accuracy: 0.9535 - val_loss: 0.1578


## Build and Train GRU Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim))
model.add(GRU(units=rnn_units))
model.add(Dense(units=num_classes, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test)
)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 54s 19ms/step - accuracy: 0.5716 - loss: 1.0894 - val_accuracy: 0.9443 - val_loss: 0.1711
Epoch 2/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 53s 19ms/step - accuracy: 0.9569 - loss: 0.1352 - val_accuracy: 0.9573 - val_loss: 0.1370
Epoch 3/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 51s 19ms/step - accuracy: 0.9713 - loss: 0.0921 - val_accuracy: 0.9522 - val_loss: 0.1509
Epoch 4/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 51s 19ms/step - accuracy: 0.9816 - loss: 0.0611 - val_accuracy: 0.9546 - val_loss: 0.1495
Epoch 5/5
2720/2720 ━━━━━━━━━━━━━━━━━━━━ 52s 19ms/step - accuracy: 0.9878 - loss: 0.0418 - val_accuracy: 0.9524 - val_loss: 0.1699
